### Import 

In [11]:
import numpy as np
import pandas as pd
import torchvision.datasets as ds
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
#Data loading
from torch.utils.data import DataLoader

### Dataset 

In [21]:

train_val_set = ds.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset = ds.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())


def to_ndarray(dataset):
    images = []
    labels = []
    
    for image, label in dataset:
        image = image.numpy().flatten()
        images.append(image)
        labels.append(label)  # Use raw labels here
    return np.array(images), np.array(labels)

X_trainval, y_trainval = to_ndarray(train_val_set)
X_test, y_test = to_ndarray(testset)

# Perform the train/validation split using raw labels
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42)

# One-hot encode labels after splitting
y_train = np.eye(10)[y_train]
y_val = np.eye(10)[y_val]
y_test = np.eye(10)[y_test]

print(X_train.shape, y_train.shape)  # Should print (48000, 784) (48000, 10)
print(X_val.shape, y_val.shape)      # Should print (12000, 784) (12000, 10)
print(X_test.shape, y_test.shape)    # Should print (10000, 784) (10000,)





(48000, 784) (48000, 10)
(12000, 784) (12000, 10)
(10000, 784) (10000, 10)


### Layer

In [ ]:
class Layer:
    def forward(self, X):
        raise NotImplementedError
    
    def backward(self, dLdy):
        raise NotImplementedError


### Dense Layer

In [ ]:
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim):
        self.weights = np.random.randn(input_dim, output_dim) * np.sqrt(2.0 / input_dim)
        self.bias = np.zeros((1, output_dim))
        self.output = None
    
    def forward(self, X):
        self.input = X
        self.output = np.dot(X, self.weights) + self.bias
        return self.output
    
    def backward(self, d_out):
        d_weights = np.dot(self.input.T, d_out)
        d_bias = np.sum(d_out, axis=0, keepdims=True)
        d_input = np.dot(d_out, self.weights.T)
        
        self.d_weights = d_weights
        self.d_bias = d_bias
        self.d_input = d_input
        return d_input


### ReLU Layer 

In [ ]:
class ReLULayer(Layer):
    def forward(self, X):
        self.input = X
        self.output = self.relu(X)
        return self.output
    
    def backward(self, d_out):
        self.d_input = d_out * np.where(self.input > 0, 1, 0)
        return self.d_input
        
    def relu(self, x):
        return np.maximum(x, 0)
    
    def relu_derivative(self, x):
        return np.where(x > 0, 1, 0)
       

### Tanh Layer

In [ ]:
class TanhLayer(Layer):
    def forward(self, X):
        self.input = X
        self.output = np.tanh(X)
        return self.output
    
    def backward(self, d_out):
        self.d_input = d_out * (1 - np.tanh(self.input) ** 2)
        return self.d_input

### Sigmoid Layer

In [ ]:
class SigmoidLayer(Layer):
    def forward(self, X):
        self.input = X
        self.output = 1 / (1 + np.exp(-X))
        return self.output
    
    def backward(self, d_out):
        self.d_input = d_out * self.output * (1 - self.output)
        return self.d_input

### Softmax Layer

In [ ]:
class SoftmaxLayer(Layer):
    def forward(self, X):
        self.input = X
        exps = np.exp(X - np.max(X, axis=1, keepdims=True))
        self.output = exps / np.sum(exps, axis=1, keepdims=True)
        return self.output
    
    def backward(self, d_out):
        return self.output - d_out
        

### Dropout Layer

In [ ]:
class DropoutLayer(Layer):
    def __init__(self, p):
        self.p = p
        self.mask = None
    
    def forward(self, X):
        self.input = X
        if self.p == 0 :
            self.output = X
            return self.output
        elif self.p == 1:
            self.output = np.zeros(X.shape)
            return self.output
        
        self.mask = np.random.binomial(1, self.p, size=X.shape) / self.p
        self.output = X * self.mask
        return self.output
    
    def backward(self, d_out):
        return d_out * self.mask

### Cross Entropy Loss

### Optimizer

### Neural Network

In [38]:

ds = Dataset()
X_train, y_train, X_val, y_val, X_test, y_test = ds.get_data()
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(48000, 784) (48000, 10)
(12000, 784) (12000, 10)
(10000, 784) (10000, 10)
